# Cálculo simbólico de derivadas com SymPy

## $ \S 1 $ Introdução

Nesta aula aprenderemos a calcular derivadas de funções usando Python.  O
cálculo de derivadas pode ser abordado de duas maneiras principais:
**simbólica** e **numérica**. Para cada tipo, usaremos um pacote diferente:
**SymPy** e **SciPy**, respectivamente. (Para distingüir um do outro,
note que "Sym" é abreviação de "Symbolic").

⚠️ Quando importados, tanto o SymPy quanto o SciPy são freqüentemente abreviados por `sp`.
Para evitar confusão, utilizaremos as abreviações abaixo.

In [2]:
import sympy as sym
import scipy as scp

O **cálculo numérico** utiliza técnicas de discretização do domínio para estimar
a derivada de uma função com base nos valores dela em alguns pontos específicos.
O *método das diferenças finitas* é o mais comumente utilizado. Na forma mais
simples, aproximamos a derivada de uma função $ f $ em $ x = x_0 $ pela
expressão
$$
f'(x_0) \approx \frac{f(x_0 + h) - f(x_0)}{h}\,
$$
para $ h \ne 0 $ pequeno. Esta é a chamada _fórmula da diferença progressiva_.
Outras fórmulas, como a da _diferença centrada_
$$
f'(x_0) \approx \frac{f(x_0 + h) - f(x_0 - h)}{2h}\,
$$
oferecem aproximações melhores. 

Infelizmente, os métodos numéricos de derivação não são muito precisos, por
exemplo quando comparados aos métodos numéricos para integração. Isto ocorre por
causa do conflito entre os erros envolvidos na interpolação em si (para os
quais a solução seria diminuir $ h $ ao máximo nas expressões acima) e os erros
de arredondamento inerentes à precisão limitada do sistema de ponto flutuante
(para os quais a solução seria não tomar $ h $ muito pequeno).

No **cálculo simbólico**, como o realizado pelo SymPy, as derivadas são obtidas
manipulando-se a expressão matemática exata da função de acordo com as regras usuais
de derivação aprendidas no curso de Cálculo. As variáveis são tratadas como
_símbolos_, e o resultado é uma expressão analítica para a derivada como uma função
independente, não apenas o valor dela num determinado ponto $ x_0 $.


|   | Cálculo Simbólico                     | Cálculo Numérico                      |
|---------------------------|---------------------------------------|---------------------------------------|
| **Precisão**              | exata                                 | aproximada                            |
| **Resultado**             | expressão analítica                   | valor numérico                        |
| **Dependência de Dados**  | não é capaz de lidar diretamente com dados        | pode ser usado com dados experimentais |
| **Aplicabilidade**        | ideal para análises teóricas          | ideal para avaliações práticas        |
| **Custo Computacional** | pode ser alto para funções complicadas | geralmente mais eficiente             |
| **Facilidade de Implementação** | requer consideração de vários casos e regras | geralmente muito simples |



## $ \S 2 $ Derivadas simbólicas de funções de uma variável

### $ 2.1 $ Exemplos

**Exemplo 1 (derivada de um polinômio):** Considere $ f(x) = x^2 + 3x + 2 $.
Vamos calcular sua derivada (em relação a $ x $).

In [2]:
# Importando o SymPy:
import sympy as sym

# Definindo a variável simbólica:
x = sym.symbols('x')

# Definindo a função:
f = x**2 + 3*x + 2

# Calculando a derivada:
df_dx = sym.diff(f, x)

# Retornando o resultado:
df_dx

2*x + 3

__Exemplo 2 (derivada da exponencial):__ Para calcular derivadas de uma função
especial, como a exponencial, o logaritmo ou o seno, precisamos importar a
versão dela fornecida pelo SymPy. Por exemplo, se $ g(t) = e^{-t^2} $:

In [3]:
# Definindo a variável simbólica:
t = sym.symbols('t')

# Definindo g:
g = sym.exp(-t * t)     # sym.exp é a exponencial

# Calculando e retornando a derivada:
dg_dt = sym.diff(g, t)
dg_dt

-2*t*exp(-t**2)

📝 Podemos utilizar qualquer caracter Unicode (ou palavra consistindo de
caracteres deste tipo) como nomes para as variáveis, por exemplo '$ \alpha $' ou
"alfa".

__Exemplo 3 (derivadas de ordem superior):__ Vamos verificar a periodicidade das
derivadas da função seno. 

In [32]:
# Definindo a variável simbólica (θ):
θ = sym.symbols('θ')

# Definindo a função seno:
f = sym.sin(θ)

# Primeira derivada:
df_dt = sym.diff(f, θ)

# Para calcular a segunda derivada, podemos derivar df_dt:
d2f_dt2 = sym.diff(df_dt, θ)
# Alternativamente, basta indicar a ordem como um argumento extra:
d2f_dt2 = sym.diff(f, θ, 2)

# Similarmente para as derivadas de ordem mais alta:
d3f_dt3 = sym.diff(f, θ, 3)
d4f_dt4 = sym.diff(f, θ, 4)

# Exibindo os resultados:
f, df_dt, d2f_dt2, d3f_dt3, d4f_dt4


(sin(θ), cos(θ), -sin(θ), -cos(θ), sin(θ))

**Exercício:** Use o SymPy para obter a primeira e segunda derivada de cada uma
das seguintes funções.

(a) $ f(x) = 5x^3 - 4x^2 + 2x - 7 \,$. Verifique ainda que $ f^{(4)} \equiv 0 $.

(b) $ g(x) = e^{3x} \sin x \,$

(c) $ h(t) = \frac{t^2 + 1}{t - 1} \,$

(d) $ u(x) = \ln(x^2 + 3x + 2) \,$ (o logaritmo natural no SymPy é `log`).

(e) $ v(\alpha) = \sqrt{\alpha^2 + \sin^2(\alpha)} \ $ (utilize a variável-símbolo '$ \alpha $').

### $ 2.2 $ Explicação do método simbólico

De maneira breve, o cálculo simbólico de derivadas consiste em formalizar
num algoritmo as regras formais de derivação, tais como:

$$
\begin{aligned}
& \big(a\, f + b\,g\big)' = a\, f' + b\, g' && (\text{regra da combinação linear}) \\
& (x^r)' = r\,x^{r - 1} && (\text{regra da potência}) \\
& (f \cdot g)' = f' \cdot g + f \cdot g' && (\text{regra do produto}) \\
& \left(\frac{f}{g}\right)' = \frac{f' \cdot g - f \cdot g'}{g^2} && (\text{regra do quociente}) \\
& (f \circ g)' = (f'\circ g) \cdot g' && (\text{regra da cadeia})
\end{aligned}
$$
Além destas, precisamos implementar diretamente a derivada de funções
especiais, como a exponencial ou o seno.

Durante o processo, o algoritmo deve simplificar automaticamente as expressões
intermediárias para tentar controlar a complexidade.  Isso envolve a combinação
de termos semelhantes, a aplicação de identidades trigonométricas, etc. Todas
estas regras precisam ser codificadas "à mão".  Apesar da implementação
trabalhosa, as idéias subjacentes a um algoritmo deste tipo são relativamente
simples.

**Exemplo 4:** Se derivarmos $ f(x) = \sin^2 x + \cos^2⁡ x $ formalmente usando
a regra da soma e a do produto, obteremos a expressão relativamente complexa
$$
f'(x) = 2\,\sin x \,\cos x + 2\,\cos x\,\big(-\sin x\big)\,.
$$
Contudo, o SymPy foi programado para notar que $ f(x) = 1 $ para todo $ x $ ou
que os dois termos na expressão para $ f' $ podem ser cancelados:

In [4]:
# Reservando o símbolo x e definindo f:
x = sym.symbols('x')
f = sym.sin(x)**2 + sym.cos(x)**2

# Calculando a derivada:
df_dx = sym.diff(f, x)
df_dx

0

__Exercício:__ O valor presente $ V $ de uma série de pagamentos mensais (por
exemplo da hipoteca de uma casa) é dado pela fórmula:
$$
V = \frac{C}{r_m} \left[1 - (1 + r_m)^{-n}\right]
$$
onde aqui:
* $ C $ é o valor do pagamento mensal;
* $ r_m $ é a taxa de juros mensal;
* $ n $ é o número total de meses da série.

Calcule a sensibilidade (ou seja, a taxa de variação) do valor presente em
relação à taxa de juros mensal.

## $ \S 3 $ Derivadas simbólicas de funções de várias variáveis

As construções da $ \S 1 $ se estendem facilmente à diferenciação simbólica de
funções de várias variáveis.

__Exemplo 1 (derivadas de uma função de duas variáveis):__
Vamos calcular as derivadas parciais com respeito a $ x $ e $ y $ da função
$ f(x, y) = x^3 - y^2 + 2xy $.

In [28]:
# Desta vez precisamos utilizar duas variáveis simbólicas:
x, y = sym.symbols('x y')

# Definindo f:
f = x**3 - y**2 + 2 * x * y

# Calculando as derivadas parciais:
df_dx = sym.diff(f, x)
df_dy = sym.diff(f, y)

# Exibindo f e suas derivadas parciais:
display(f, df_dx, df_dy)

x**3 + 2*x*y - y**2

3*x**2 + 2*y

2*x - 2*y

📝 A função `display` exibe os resultados num formato mais amigável e legível
(usando $ \LaTeX $). Ela tem boa integração com o SymPy e está automaticamente
disponível em cadernos Jupyter, mas fora deles precisa ser importada com:
`from Ipython.display import display`.

__Exemplo 2 (derivadas de ordem superior):__ 
Calcule as derivadas parciais de primeira e segunda ordem da função $ g(u,v) = \sin⁡(uv) $.

In [5]:
# Definindo as variáveis e a função g: 
u, v = sym.symbols('u v')
g = sym.sin(u * v)

# Calculando as derivadas parciais de primeira ordem:
dg_du = sym.diff(g, u)
dg_dv = sym.diff(g, v)

# Para calcular a derivada parcial de ordem 2 com respeito a u,
# podemos derivar dg_du com respeito a u:
d2g_du2 = sym.diff(dg_du, u)
# Porém é mais fácil e natural derivar g com respeito a u e a u:
d2g_du2 = sym.diff(g, u, u)
# Ou ainda: 
d2g_du2 = sym.diff(g, u, 2)

# Similarmente para a derivada parcial com respeito a v:
d2g_dv2 = sym.diff(g, v, 2)

# Agora a derivada parcial mista:
d2g_dudv = sym.diff(g, u, v)

# Exibindo os resultados:
display(g, dg_du, dg_dv, d2g_du2, d2g_dv2, d2g_dudv)

sin(u*v)

v*cos(u*v)

u*cos(u*v)

-v**2*sin(u*v)

-u**2*sin(u*v)

-u*v*sin(u*v) + cos(u*v)

__ExercÍcio (função de três variáveis):__ Determine todas as derivadas parciais
de primeira e segunda ordem da função $$ h(x,y,z) = x^2 y + yze^z\,. $$

__Exercício:__

(a) Quantas derivadas parciais de ordem $ 3 $ tem uma função de duas variáveis?

(b) Quantas derivadas parciais de ordem $ r $ tem uma função $ f $ de $ m $ variáveis?
Você pode assumir que a ordem em que tomamos derivadas parciais não importa, de modo
que por exemplo:
$$
\frac{\partial^3 f}{\partial x\, \partial y^2} = \frac{\partial^3 f}{\partial y\, \partial x\, \partial y}
$$
_Dica:_ Este problema é equivalente ao seguinte: de quantas maneiras podemos
alocar $ r $ bolas indistingüíveis em $ m $ caixas distintas?

## $ \S 4 $ Calculando o gradiente

### $ 4.1 $ O gradiente de funções de duas variáveis

Seja $ z = f(x, y) $ uma função de duas variáveis $ x $ e $ y $. O __gradiente__ de
$ f $, denotado por $ \nabla f $, é um campo vetorial cujas componentes são as
duas derivadas parciais de $ f $:
$$
\nabla f = \big(f_x\,,\,f_y\big) = \bigg(
\frac{\partial f}{\partial x}\,,\, \frac{\partial f}{\partial y}
\bigg)\,.
$$

Em um ponto específico do domínio, o gradiente aponta na direção de maior
crescimento da função, e a taxa de maior crescimento é dada pela norma
(magnitude) do gradiente.

Geometricamente, o gradiente de uma função $ f $ em cada ponto é perpendicular
à __curva de nível__ de $ f $ passando por aquele ponto. A curva de nível $ L_c
$ correspondente a $ z = c $ é, por definição, o subconjunto de pontos do
domínio onde $ f $ vale $ c $, ou seja, $ L_c = f^{-1}(c) $.

No SymPy, podemos calcular o gradiente de uma função utilizando o procedimento
`derive_by_array`. Mais precisamente, ele permite calcular as derivadas parciais
de uma função com respeito a um conjunto qualquer de variáveis,
retornando um array (do SymPy, não do NumPy) como resultado.

__Exemplo 1:__ Vamos determinar o gradiente da função $ f(x, y) = x^2 + xy - y^2\, $.

In [6]:
# Definindo as variáveis simbólicas e f:
x, y = sym.symbols('x y')
f = x**2 + x*y - y**2

# Calculando o gradiente usando `derive_by_array`:
grad_f = sym.derive_by_array(f, (x, y))

# Exibindo os resultados:
display(f, grad_f)

# Checando que o resultado é um array:
print(type(grad_f))

x**2 + x*y - y**2

[2*x + y, x - 2*y]

<class 'sympy.tensor.array.dense_ndim_array.ImmutableDenseNDimArray'>


![Curvas de nível e gradiente](curvas_de_nivel_gradiente.png)

### $ 4.2 $ O gradiente de funções de várias variáveis

Seja $ w = f(x_1, x_2, \ldots, x_m) $ uma função de $ m $ variáveis, definida num subconjunto de $ \mathbb R^m $. Por definição, o **gradiente** de $ f $ é dado por
$$
\nabla f = \big(f_{x_1}\,,\,f_{x_2}\,,\, \cdots \,,\, f_{x_m}\big) = \bigg( \frac{\partial f}{\partial x_1}\,,\, \frac{\partial f}{\partial x_2}\,,\, \ldots\,,\, \frac{\partial f}{\partial x_m} \bigg)\,.
$$

Assim como no caso de duas variáveis, o gradiente aponta na direção de maior
crescimento da função e sua magnitude representa a taxa máxima de variação nesse
ponto. Além disto, o gradiente é perpendicular aos **conjuntos de nível**
de $ f $. Um conjunto de nível $ L_c $ correspondente a $ w = c $ é o conjunto
de pontos do domínio onde $ f $ assume o valor $ c $, isto é, $ L_c = f^{-1}(c)
$.  Para a maioria dos valores de $ c $, $ L_c $ forma uma hipersuperfície
de dimensão $ m - 1 $ dentro do domínio de $ f $.

Um __ponto crítico__ de $ f $ é um ponto $ \mathbf p $ de seu domínio onde o
gradiente se anula (é igual ao vetor nulo de $ \mathbb R^m $); em símbolos,
$$
\nabla f(\mathbf p) = \mathbf 0 = \big(0, 0, \cdots, 0)\,.
$$
Os pontos críticos são os candidatos a mínimo e máximo local de $ f $ _no
interior do domínio_.  Mais precisamente, qualquer mínimo ou máximo local contido
no interior do domínio tem de ser um ponto crítico, mas nem todo ponto crítico é
necessariamente um extremo local.

__Exemplo:__ Vamos encontrar os pontos críticos da função $ g(x, y, z) = x^2 +
y^2 + z^2 - 4xz + 2y $ definida em todo o $ \mathbb R^3 $. Começamos calculando
o gradiente como antes:

In [13]:
# Definindo as variáveis e a função g:
x, y, z = sym.symbols('x y z')

g = x**2 + y**2 + z**2 - 4*x*z + 2*y

# Calculando o gradiente:
grad_g = sym.derive_by_array(g, (x, y, z))
grad_g

[2*x - 4*z, 2*y + 2, -4*x + 2*z]

Neste caso, igualando o gradiente ao vetor nulo, obtemos um sistema linear de
três equações nas três incógnitas $ x $, $ y $ e $ z $ que pode facilmente ser
resolvido à mão. A única solução é $ (0, -1, 0) $. Mas também podemos delegar
esta parte do trabalho ao computador usando a função `sympy.solve` como abaixo:

In [23]:
# Encontrando os pontos críticos (gradiente igual a zero):
critical_points = sym.solve(grad_g, (x, y, z))

# Exibindo a solução:
critical_points

{x: 0, y: -1, z: 0}

📝 `sympy.solve` tenta encontrar os zeros _exatos_ de uma função usando técnicas
algébricas e outras transformações. Se esta solução não puder ser encontrada,
o resultado pode ser uma expressão implícita ou paramétrica, ou o SymPy pode
gerar apenas um número finito de zeros e ignorar outros.  Para obter
_aproximações numéricas_ para a solução, utilize o procedimento `sympy.nsolve`.

__Exercício:__ Para cada uma das funções abaixo, determine o gradiente
usando `sympy.derive_by_array` e encontre os pontos críticos resolvendo o
sistema de equações formado ao se igualar o gradiente ao vetor nulo com
ajuda do procedimento `sympy.solve`.

(a) $ f(x, y) = x^3 - 3xy^2 $.

(b) $ g(x, y) = \sin(x) \sin(y) $. As soluções fornecidas pelo `solve` são exaustivas?

(c) $ h(x, y, z) = x^2 + y^2 + z^2 - 4xz + 2y $.